In [ ]:
import cv2

def img_to_vect(img):
    pass

def graficar_vector_imagen(x, img):
    pass

def descorrelacion(x):
    pass

def graficar_vector(y, lista):
    pass

#Imagen1
img1 = cv2.imread('img_01.jpg', cv2.IMREAD_GRAYSCALE)
x1 = img_to_vect(img1)
graficar_vector_imagen(x1, img1)

#Imagen2
img2 = cv2.imread('img_02.jpg', cv2.IMREAD_GRAYSCALE)
x2 = img_to_vect(img2)
graficar_vector_imagen(x2, img2)

#Descorrelacion del vector X (PCA sin reducir dimension)
y1 = descorrelacion(x1)
y2 = descorrelacion(x2)

#Graficar vectores proyectados
graficar_vector(y1, ['Y1', 'Y2'])
graficar_vector(y2, ['Y1', 'Y2'])




hola
